# Das Rucksackproblem

Das Rucksackproblem ist ein klassisches Problem der Kombinatorik und der Informatik. Wir haben eine Menge von Gegenständen, die wir in einen Rucksack packen wollen. Jeder Gegenstand hat ein Gewicht und einen Wert. Das Ziel ist es, den Rucksack so zu packen, dass der Wert der Gegenstände im Rucksack maximal ist, ohne dass das Gesamtgewicht der Gegenständen die Kapazität überschreitet.

![Rucksack](https://upload.wikimedia.org/wikipedia/commons/f/fd/Knapsack.svg)

## Das mathematische Modell

Mit $n$ Gegenständen, die wir in den Rucksack packen können, und einer Kapazität $W$ des Rucksacks, können wir das Rucksackproblem als ein ganzzahliges lineares Programm formulieren:

$$
\begin{align*}
\max \quad & \sum_{i=1}^{n} v_i x_i \\
\text{unter den Nebenbedingungen} \quad & \sum_{i=1}^{n} w_i x_i \leq W \\
& x_i \in \{0, 1\} \quad \text{für alle } i = 1, \ldots, n
\end{align*}
$$

Die Entscheidungsvariablen $x_i$ geben an, ob der Gegenstand $i$ in den Rucksack gepackt wird ($x_i = 1$) oder nicht ($x_i = 0$). Der Wert des Gegenstands $i$ ist $v_i$ und sein Gewicht ist $w_i$. Sowohl der Wert als auch das Gewicht sind bekannt und streng positiv.

In [9]:
items_value = [20, 50, 23, 25]
items_weight = [2, 3, 1, 4]
max_weight = 5

chosen = [0, 1, 1, 0]

value_of_chosen = sum([a*b for a, b in zip(items_value, chosen)])
weight_of_chosen = sum([a*b for a, b in zip(items_weight, chosen)])

print("Value of chosen items: ", value_of_chosen)
print("Weight of chosen items: ", weight_of_chosen)
print("Max weight: ", max_weight)
print("Chosen items: ", chosen)

Value of chosen items:  73
Weight of chosen items:  4
Max weight:  5
Chosen items:  [0, 1, 1, 0]


In [4]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
items_value = [20, 50, 23, 12]
items_weight = [2, 3, 1, 4]
max_weight = 30

m = gp.Model("knapsack")

x = m.addVars(len(items_value), vtype=GRB.BINARY, name="x")

m.setObjective(sum(items_value[i] * x[i] for i in range(len(items_value))), GRB.MAXIMIZE)

m.addConstr(sum(items_weight[i] * x[i] for i in range(len(items_weight))) <= max_weight)

m.optimize()

print("Optimal solution")

chosen_items = []

for v in m.getVars():
    if v.x > 0.1:
        chosen_items.append(v.varName)

print("Chosen items: ", chosen_items)
print("Total value: ", m.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xdbaf241c
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 105.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 105 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.050000000000e+02, best bound 1.050000000000e+02, gap 0.0000%
Optimal solution
Chosen items:  ['x[0]', 'x[1]', 'x[2]', 'x[3]']
Total value:  105.0


In [3]:
def knapsack(items_value, items_weight, max_weight, max_time=60):
    m = gp.Model("knapsack")
    m.params.TimeLimit = max_time
    m.params.LogToConsole = True

    x = m.addVars(len(items_value), vtype=GRB.BINARY, name="x")

    m.setObjective(sum(items_value[i] * x[i] for i in range(len(items_value))), GRB.MAXIMIZE)

    m.addConstr(sum(items_weight[i] * x[i] for i in range(len(items_weight))) <= max_weight)

    m.optimize()

    print("Optimal solution")

    chosen_items = []

    for v in m.getVars():
        if v.x > 0.1:
            chosen_items.append(v.varName)

    print("Chosen items: ", chosen_items)
    print("Total value: ", m.objVal)

    return chosen_items, m.objVal

In [22]:
n = 2000

items_value = np.random.uniform(1, 100, n)
items_weight = np.random.uniform(1, 100, n)
max_weight = np.sum(items_weight) / 6

knapsack(items_value, items_weight, max_weight)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 2000 columns and 2000 nonzeros
Model fingerprint: 0xad2d58a3
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 2e+04]
Found heuristic solution: objective 17103.073773
Presolve time: 0.00s
Presolved: 1 rows, 2000 columns, 2000 nonzeros
Variable types: 0 continuous, 2000 integer (2000 binary)
Found heuristic solution: objective 27352.731522

Root relaxation: objective 4.686235e+04, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

(['x[0]',
  'x[3]',
  'x[5]',
  'x[6]',
  'x[7]',
  'x[10]',
  'x[11]',
  'x[12]',
  'x[15]',
  'x[17]',
  'x[20]',
  'x[21]',
  'x[22]',
  'x[26]',
  'x[27]',
  'x[28]',
  'x[30]',
  'x[37]',
  'x[38]',
  'x[40]',
  'x[41]',
  'x[47]',
  'x[53]',
  'x[57]',
  'x[59]',
  'x[60]',
  'x[61]',
  'x[63]',
  'x[69]',
  'x[71]',
  'x[78]',
  'x[87]',
  'x[91]',
  'x[97]',
  'x[100]',
  'x[102]',
  'x[103]',
  'x[107]',
  'x[113]',
  'x[114]',
  'x[117]',
  'x[119]',
  'x[121]',
  'x[126]',
  'x[130]',
  'x[131]',
  'x[135]',
  'x[137]',
  'x[139]',
  'x[140]',
  'x[144]',
  'x[149]',
  'x[150]',
  'x[154]',
  'x[160]',
  'x[162]',
  'x[170]',
  'x[172]',
  'x[176]',
  'x[181]',
  'x[184]',
  'x[188]',
  'x[206]',
  'x[207]',
  'x[208]',
  'x[210]',
  'x[212]',
  'x[213]',
  'x[215]',
  'x[229]',
  'x[235]',
  'x[242]',
  'x[244]',
  'x[245]',
  'x[248]',
  'x[251]',
  'x[257]',
  'x[258]',
  'x[259]',
  'x[260]',
  'x[262]',
  'x[264]',
  'x[269]',
  'x[271]',
  'x[275]',
  'x[278]',
  'x[27